### 공동주택 코드 매핑

In [197]:
# 라이브러리
import os
import chardet
import pandas as pd
import re 

p = re.compile('\(([^)]+)')  

def jibun_juso(addr):
    d = p.findall(addr) 
    if isinstance(d, list) and len(d) > 0:
        return d[0]
    return None

def doro_juso(addr):
    p = addr.find("(")
    juso = addr[0:p-1].rstrip()
    return juso

def part_back(obj):
    # 데이터에 null 있음    
    if isinstance(obj, str) and len(obj) > 0:
        part_front = obj.split()[0]
        part_back = obj.replace(part_front, "").lstrip()
        return part_back
    else:
        return None
    
def part_front(obj):
    part_front = obj[1].replace(obj[0], "").rstrip()
    return part_front

def no_blank(obj):
    # 데이터에 null 있음    
    if isinstance(obj, str) and len(obj) > 0:
        re_str = obj.replace(" ", "")
        return re_str
    else:
        return None

In [198]:
# 데이터 파일 위치
data_root = "D:\PythonProject\data-gatherer\molit_apt\data"
# 아파트 코드 목록: 외부(국토교통부)
file_ext = os.path.join(data_root, "공동주택_국토부_apt_info_20240411.csv")
# 아파트 코드 목록: 내부(고객지원시스템)
file_int = os.path.join(data_root, "공동주택_고객지원시스템_20240415.csv")

# 파일의 인코딩 확인
with open(file_ext, 'rb') as f:
    rawdata = f.read()
    result = chardet.detect(rawdata)
    encoding_ext = result['encoding']
    
with open(file_int, 'rb') as f:
    rawdata = f.read()
    result = chardet.detect(rawdata)
    encoding_int = result['encoding']

* df_int

In [199]:
# 파일 데이터 읽기
data_int = pd.read_csv(file_int, encoding=encoding_int, dtype=object)
df_int = data_int.copy()
int_cnt = df_int.shape[0]

In [200]:
""" 데이터 정리 """
# 1. 공동주택이 아닌 데이터 제외
idx = df_int[df_int['용도'] != '공동주택'].index
df_int.drop(idx, inplace=True)
# 2. 필요한 항목만 남김
df_int = df_int[['아파트명', '아파트코드', '우편번호', '주소']]
# 3. 중복 데이터 없애기
df_int.drop_duplicates(subset=None, keep='first', inplace=True, ignore_index=True)
# 4. 시군구 항목 생성
df_int['시군구'] = df_int['주소'].apply(lambda x: x.split()[0])
# 5. 대구광역시 공급권역 데이터만 남기고 나머지(경북 지역) 데이터 삭제
idx = df_int[~df_int['시군구'].isin(['중구', '동구', '서구', '남구', '북구', '수성구', '달서구', '달성군'])].index
df_int.drop(idx, inplace=True)
# 6. 지번주소 항목 분리 생성
df_int['지번주소'] = df_int['주소'].apply(jibun_juso)
df_int['지번주소_공백X'] = df_int['지번주소'].apply(no_blank)
# 7. 도로명주소 항목 분리 생성
df_int['시군구_도로명주소'] = df_int['주소'].apply(doro_juso)
df_int['도로명주소'] = df_int['시군구_도로명주소'].apply(part_back)
df_int['도로명주소_공백X'] = df_int['도로명주소'].apply(no_blank)
# 8. 아파트명 공백X
df_int['아파트명_공백X'] = df_int['아파트명'].apply(lambda x: x.replace(" ",""))
# 9. 필요항 컬럼만 구성
df_int = df_int[['아파트코드','아파트명', '아파트명_공백X', '시군구', '지번주소', '지번주소_공백X', '도로명주소', '도로명주소_공백X']]

* df_ext

In [201]:
# 파일 데이터 읽기
data_ext = pd.read_csv(file_ext, encoding=encoding_ext, dtype=object)
df_ext = data_ext.copy()
ext_cnt = df_ext.shape[0]

In [202]:
# df_ext.isnull().sum()

In [203]:
""" 데이터 정리 """
# 1. 필요한 항목만 남김
df_ext = df_ext[['kapt_cd', 'kapt_nm', 'as2', 'as3', 'as4', 'kapt_addr', 'doro_juso']]
# 2. 시군구_도로명주소 항목 생성
df_ext['시군구_도로명주소'] = df_ext['doro_juso'].str.lstrip('대구광역시')
# 3. 도로명주소 항목 생성
df_ext['도로명주소'] = df_ext['시군구_도로명주소'].apply(part_back)
df_ext['도로명주소_공백X'] = df_ext['도로명주소'].apply(no_blank)
# 4. 법정동주소 항목 생성   
df_ext['광역시_법정동주소'] = df_ext[['kapt_nm', 'kapt_addr']].apply(part_front, axis=1)
df_ext['시군구_법정동주소'] = df_ext['광역시_법정동주소'].apply(part_back)
df_ext['법정동주소'] = df_ext['시군구_법정동주소'].apply(part_back)
df_ext['법정동주소_공백X'] = df_ext['법정동주소'].apply(no_blank)
# 5. 컬럼명 변경
df_ext.rename(columns={'kapt_nm' : '아파트명', 'as2' : '시군구'}, inplace=True)
# 6. 아파트명 공백X
df_ext['아파트명_공백X'] = df_ext['아파트명'].apply(lambda x: x.replace(" ",""))
# 7. 필요한 컬럼만 구성
df_ext = df_ext[['kapt_cd', '아파트명', '아파트명_공백X', '시군구', 'as3', 'as4', '도로명주소', '도로명주소_공백X', '법정동주소', '법정동주소_공백X']]

* 1차 매칭 - 도로명주소

In [204]:
# 1차 매칭
df_merge1 = pd.merge(df_ext, df_int, how='outer', left_on="도로명주소_공백X", right_on="도로명주소_공백X", suffixes=('_left','_right'), indicator=True)
df_match1 = df_merge1[df_merge1['_merge'] == 'both'][['kapt_cd', '아파트코드']]
df_a = df_match1[df_match1.duplicated(subset='kapt_cd', keep=False)]       # N:1 Data
df_b = df_match1[df_match1.duplicated(subset='아파트코드', keep=False)]     # 1:N Data
df_dup = pd.concat([df_a, df_b])
idx_dup = df_match1[df_match1.index.isin(df_dup.index)].index
df_match1.drop(idx_dup, inplace=True)       # 데이터 삭제 (abnormal matching)
df_match1['match'] = '1. 도로명주소 일치'
# 1차 매칭된 데이터 삭제
idx_ext = df_ext[df_ext['kapt_cd'].isin(df_match1['kapt_cd'])].index
df_ext.drop(idx_ext, inplace=True)
idx_int = df_int[df_int['아파트코드'].isin(df_match1['아파트코드'])].index
df_int.drop(idx_int, inplace=True)
assert df_ext.shape[0] == ext_cnt - df_match1.shape[0]

* 2차 매칭 - 아파트명

In [205]:
# 2차 매칭
df_merge2 = pd.merge(df_ext, df_int, how='outer', left_on="아파트명_공백X", right_on="아파트명_공백X", suffixes=('_left','_right'), indicator=True)
df_match2 = df_merge2[df_merge2['_merge'] == 'both'][['kapt_cd', '아파트코드']]
df_a = df_match2[df_match2.duplicated(subset='kapt_cd', keep=False)]       # N:1 Data
df_b = df_match2[df_match2.duplicated(subset='아파트코드', keep=False)]     # 1:N Data
df_dup = pd.concat([df_a, df_b])
idx_dup = df_match2[df_match2.index.isin(df_dup.index)].index
df_match2.drop(idx_dup, inplace=True)       # 데이터 삭제 (abnormal matching)
df_match2['match'] = '2. 아파트명 일치'
# 2차 매칭된 데이터 삭제
idx_ext = df_ext[df_ext['kapt_cd'].isin(df_match2['kapt_cd'])].index
df_ext.drop(idx_ext, inplace=True)
idx_int = df_int[df_int['아파트코드'].isin(df_match2['아파트코드'])].index
df_int.drop(idx_int, inplace=True)
assert df_ext.shape[0] == ext_cnt - df_match1.shape[0] - df_match2.shape[0]

* 3차 매칭 - [EXT]법정동주소 == [INT]지번주소

In [206]:
# 3차 매칭
df_merge3 = pd.merge(df_ext, df_int, how='outer', left_on="법정동주소_공백X", right_on="지번주소_공백X", suffixes=('_left','_right'), indicator=True)
df_match3 = df_merge3[df_merge3['_merge'] == 'both'][['kapt_cd', '아파트코드']]
df_a = df_match3[df_match3.duplicated(subset='kapt_cd', keep=False)]       # N:1 Data
df_b = df_match3[df_match3.duplicated(subset='아파트코드', keep=False)]     # 1:N Data
df_dup = pd.concat([df_a, df_b])
idx_dup = df_match3[df_match3.index.isin(df_dup.index)].index
df_match3.drop(idx_dup, inplace=True)       # 데이터 삭제 (abnormal matching)
df_match3['match'] = '3. 국토부_법정동주소와 내부시스템_지번주소 일치'
# 3차 매칭된 데이터 삭제
idx_ext = df_ext[df_ext['kapt_cd'].isin(df_match3['kapt_cd'])].index
df_ext.drop(idx_ext, inplace=True)
idx_int = df_int[df_int['아파트코드'].isin(df_match3['아파트코드'])].index
df_int.drop(idx_int, inplace=True)
assert df_ext.shape[0] == ext_cnt - df_match1.shape[0] - df_match2.shape[0] - df_match3.shape[0]

* 4차 매칭

In [207]:
# 4차 매칭
df_ext['as3_도로명주소_공백X'] = df_ext['as3'] + df_ext['도로명주소_공백X']
df_merge4 = pd.merge(df_ext, df_int, how='outer', left_on="as3_도로명주소_공백X", right_on="도로명주소_공백X", suffixes=('_left','_right'), indicator=True)
df_match4 = df_merge4[df_merge4['_merge'] == 'both'][['kapt_cd', '아파트코드']]
df_a = df_match4[df_match4.duplicated(subset='kapt_cd', keep=False)]       # N:1 Data
df_b = df_match4[df_match4.duplicated(subset='아파트코드', keep=False)]     # 1:N Data
df_dup = pd.concat([df_a, df_b])
idx_dup = df_match4[df_match4.index.isin(df_dup.index)].index
df_match4.drop(idx_dup, inplace=True)       # 데이터 삭제 (abnormal matching)
df_match4['match'] = '4. 국토부_읍면+도로명주소와 내부시스템_도로명주소 일치'
# 4차 매칭된 데이터 삭제
idx_ext = df_ext[df_ext['kapt_cd'].isin(df_match4['kapt_cd'])].index
df_ext.drop(idx_ext, inplace=True)
idx_int = df_int[df_int['아파트코드'].isin(df_match4['아파트코드'])].index
df_int.drop(idx_int, inplace=True)
assert df_ext.shape[0] == ext_cnt - df_match1.shape[0] - df_match2.shape[0] - df_match3.shape[0] - df_match4.shape[0]

* 5차 매칭

In [208]:
same_char = {"LH" : "엘에이치", "e" : "이"}
def re_char(apt_name):
    y = apt_name
    for k, v in same_char.items():
        if k in apt_name:
            y = apt_name.replace(k, v)
    return y

# 5차 매칭
df_ext['아파트명_공백X_단어통일'] = df_ext['아파트명_공백X'].apply(re_char)
df_int['아파트명_공백X_단어통일'] = df_int['아파트명_공백X'].apply(re_char)
df_merge5 = pd.merge(df_ext, df_int, how='outer', left_on="아파트명_공백X_단어통일", right_on="아파트명_공백X_단어통일", suffixes=('_left','_right'), indicator=True)
df_match5 = df_merge5[df_merge5['_merge'] == 'both'][['kapt_cd', '아파트코드']]
df_a = df_match5[df_match5.duplicated(subset='kapt_cd', keep=False)]       # N:1 Data
df_b = df_match5[df_match5.duplicated(subset='아파트코드', keep=False)]     # 1:N Data
df_dup = pd.concat([df_a, df_b])
idx_dup = df_match5[df_match5.index.isin(df_dup.index)].index
df_match5.drop(idx_dup, inplace=True)       # 데이터 삭제 (abnormal matching)
df_match5['match'] = '5. 아파트명 유사단어 통일 후 일치'
# 5차 매칭된 데이터 삭제
idx_ext = df_ext[df_ext['kapt_cd'].isin(df_match5['kapt_cd'])].index
df_ext.drop(idx_ext, inplace=True)
idx_int = df_int[df_int['아파트코드'].isin(df_match5['아파트코드'])].index
df_int.drop(idx_int, inplace=True)   
assert df_ext.shape[0] == ext_cnt - df_match1.shape[0] - df_match2.shape[0] - df_match3.shape[0] - df_match4.shape[0] - df_match5.shape[0]

* 6차 매칭

In [209]:
extra_char = ['대구', '아파트']
def del_char(apt_name):
    y = apt_name
    for c in extra_char:
        if c in y:
            y = y.replace(c, "")     # 해당 단어 삭제
    return y

# 6차 매칭
df_ext['아파트명_공백X_부가단어삭제'] = df_ext['아파트명_공백X'].apply(del_char)
df_int['아파트명_공백X_부가단어삭제'] = df_int['아파트명_공백X'].apply(del_char)
df_merge6 = pd.merge(df_ext, df_int, how='outer', left_on="아파트명_공백X_부가단어삭제", right_on="아파트명_공백X_부가단어삭제", suffixes=('_left','_right'), indicator=True)
df_match6 = df_merge6[df_merge6['_merge'] == 'both'][['kapt_cd', '아파트코드']]
df_a = df_match6[df_match6.duplicated(subset='kapt_cd', keep=False)]       # N:1 Data
df_b = df_match6[df_match6.duplicated(subset='아파트코드', keep=False)]     # 1:N Data
df_dup = pd.concat([df_a, df_b])
idx_dup = df_match6[df_match6.index.isin(df_dup.index)].index
df_match6.drop(idx_dup, inplace=True)       # 데이터 삭제 (abnormal matching)
df_match6['match'] = '6. 아파트명 부가단어 삭제 후 일치'
# 6차 매칭된 데이터 삭제
idx_ext = df_ext[df_ext['kapt_cd'].isin(df_match6['kapt_cd'])].index
df_ext.drop(idx_ext, inplace=True)
idx_int = df_int[df_int['아파트코드'].isin(df_match6['아파트코드'])].index
df_int.drop(idx_int, inplace=True)   
assert df_ext.shape[0] == ext_cnt - df_match1.shape[0] - df_match2.shape[0] - df_match3.shape[0] - df_match4.shape[0] - df_match5.shape[0] - df_match6.shape[0]

* 총합

In [210]:
m1 = df_match1.shape[0]
m2 = df_match2.shape[0]
m3 = df_match3.shape[0]
m4 = df_match4.shape[0]
m5 = df_match5.shape[0]
m6 = df_match6.shape[0]
total = m1 + m2 + m3 + m4 + m5 + m6
print(f"대구광역시 국토부 데이터:\t {ext_cnt}")
print(f"고객지원시스템 데이터:\t {int_cnt}")
print(f"매칭: 1차 매칭 {m1}, 2차 매칭 {m2}, 3차 매칭 {m3}, 4차 매칭 {m4}, 5차 매칭 {m5}, 6차 매칭 {m6}")
print(f"총 매칭 {total}, 잔여 건수(국토부 데이터 기준) {ext_cnt - total}")

대구광역시 국토부 데이터:	 1039
고객지원시스템 데이터:	 3170
매칭: 1차 매칭 831, 2차 매칭 106, 3차 매칭 29, 4차 매칭 43, 5차 매칭 4, 6차 매칭 6
총 매칭 1019, 잔여 건수(국토부 데이터 기준) 20


In [211]:
# 매칭 데이터 병합
df_list = [df_match1, df_match2, df_match3, df_match4, df_match5, df_match6]
df_all = pd.concat(df_list, ignore_index=True)
df_all.sort_values(by='match')['match'].value_counts()

match
1. 도로명주소 일치                        831
2. 아파트명 일치                         106
4. 국토부_읍면+도로명주소와 내부시스템_도로명주소 일치     43
3. 국토부_법정동주소와 내부시스템_지번주소 일치         29
6. 아파트명 부가단어 삭제 후 일치                 6
5. 아파트명 유사단어 통일 후 일치                 4
Name: count, dtype: int64

* 결과 파일 저장

In [218]:
data_all = df_all.copy()
data_all = pd.merge(data_all, data_ext[['kapt_cd','kapt_nm']], how='left', left_on='kapt_cd', right_on='kapt_cd') 
data_all = pd.merge(data_all, data_int[['아파트코드','아파트명']], how='left', left_on='아파트코드', right_on='아파트코드') 

In [226]:
import xlsxwriter
#1. 파일 생성
file_matching = os.path.join(data_root, "공동주택_매칭결과.xlsx")
writer = pd.ExcelWriter(file_matching, engine='xlsxwriter')
 
with pd.ExcelWriter(file_matching) as writer:
    data_all.to_excel(writer, sheet_name='Matched')
    df_ext.to_excel(writer, sheet_name='Unmatched_국토부')
    df_int.to_excel(writer, sheet_name='Unmatched_고객지원시스템')